##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Entrenamiento multitrabajador con Estimator

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

> Advertencia: Los estimadores no se recomiendan para código nuevo. Los estimadores ejecutan el código de estilo `v1.Session` que es más difícil de escribir correctamente y puede tener un comportamiento inesperado; particularmente, cuando se combina con código TF 2. Los estimadores están incluidos dentro de nuestras [garantías de compatibilidad](https://tensorflow.org/guide/versions), pero no se les harán correcciones a menos que se trate de vulnerabilidades de seguridad. Para más detalles, consulte la [Guía de migración](https://tensorflow.org/guide/migrate).

## Visión general

Nota: Aunque puede usar Estimators con la API `tf.distribute`, se recomienda usar Keras con `tf.distribute`, consulte [entrenamiento multitrabajador con Keras](multi_worker_with_keras.ipynb). El entrenamiento de estimadores con `tf.distribute.Strategy` tiene un soporte limitado.

Este tutorial demuestra cómo se puede usar `tf.distribute.Strategy` para el entrenamiento distribuido multitrabajador con `tf.estimator`. Si escribe su código usando `tf.estimator`, y está interesado en escalar más allá de una sola máquina con alto rendimiento, este tutorial es para usted.

Antes de empezar, lea la guía [estrategia de distribución](../../guide/distributed_training.ipynb). El [tutorial de entrenamiento multi-GPU](./keras.ipynb) también es relevante, porque este tutorial usa el mismo modelo.


## Preparación

Primero, configure TensorFlow y las importaciones necesarias.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os, json

Nota: A partir de TF2.4 la estrategia multitrabajador en espejo falla con los estimadores si se ejecuta con eager habilitado (la opción predeterminada). El error en TF2.4 es `TypeError: cannot pickle '_thread.lock' object`, Véase [problema #46556](https://github.com/tensorflow/tensorflow/issues/46556) para más detalles. La solución consiste en desactivar eager execution.

In [ ]:
tf.compat.v1.disable_eager_execution()

## Función de entrada

Este tutorial usa el conjunto de datos MNIST de [Conjuntos de datos de TensorFlow](https://www.tensorflow.org/datasets).  El código aquí es similar al del [tutorial de entrenamiento multi-GPU](./keras.ipynb) con una diferencia clave: cuando se usa Estimator para el entrenamiento multitrabajador, es necesario fragmentar el conjunto de datos según el número de trabajadores para asegurar la convergencia del modelo.  Los datos de entrada se fragmentan según el índice de trabajadores, de modo que cada trabajador procesa `1/num_workers` porciones distintas del conjunto de datos.

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def input_fn(mode, input_context=None):
  datasets, info = tfds.load(name='mnist',
                                with_info=True,
                                as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else
                   datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  if input_context:
    mnist_dataset = mnist_dataset.shard(input_context.num_input_pipelines,
                                        input_context.input_pipeline_id)
  return mnist_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Otro enfoque razonable para lograr la convergencia sería barajar el conjunto de datos con semillas distintas en cada trabajador.

## Configuración multitrabajador

Una de las diferencias clave en este tutorial (en comparación con el tutorial de entrenamiento [multi-GPU](./keras.ipynb)) es la configuración multitrabajador.  La variable de entorno `TF_CONFIG` es la forma estándar de especificar la configuración del cluster a cada trabajador que forma parte del cluster.

Hay dos componentes de `TF_CONFIG`: `cluster` y `task`. `cluster` ofrece información sobre todo el clúster, es decir, los trabajadores y los servidores de parámetros del clúster. `task` ofrece información sobre la tarea actual. El primer componente `cluster` es el mismo para todos los trabajadores y servidores de parámetros del cluster, y el segundo componente `task` es diferente en cada trabajador y servidor de parámetros y especifica su propio `type` y `index`. En este ejemplo, la tarea `type` es `worker` y la tarea `index` es `0`.

A modo de ilustración, este tutorial muestra cómo configurar un `TF_CONFIG` con 2 trabajadores en `localhost`. En la práctica, usted crearía varios trabajadores en una dirección IP y un puerto externos, y configuraría `TF_CONFIG` en cada trabajador adecuadamente, es decir, modificaría la tarea `index`.

Advertencia: *No ejecute el siguiente código en Colab.* El runtime de TensorFlow intentará crear un servidor gRPC en la dirección IP y puerto especificados, lo que probablemente fracasará. Consulte la [versión con keras](multi_worker_with_keras.ipynb) de este tutorial para un ejemplo de cómo puede probar la ejecución de múltiples trabajadores en una sola máquina.

```
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})
```


## Definir el modelo

Escriba las capas, el optimizador y la función de pérdida para el entrenamiento. Este tutorial define el modelo con capas Keras, similar al [tutorial de entrenamiento multi-GPU](./keras.ipynb).

In [ ]:
LEARNING_RATE = 1e-4
def model_fn(features, labels, mode):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  logits = model(features, training=False)

  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {'logits': logits}
    return tf.estimator.EstimatorSpec(labels=labels, predictions=predictions)

  optimizer = tf.compat.v1.train.GradientDescentOptimizer(
      learning_rate=LEARNING_RATE)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(labels, logits)
  loss = tf.reduce_sum(loss) * (1. / BATCH_SIZE)
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss)

  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=optimizer.minimize(
          loss, tf.compat.v1.train.get_or_create_global_step()))

Nota: Aunque la tasa de aprendizaje es fija en este ejemplo, en general puede ser necesario ajustarla en función del tamaño global del lote.

## MultiWorkerMirroredStrategy

Para entrenar el modelo, use una instancia de `tf.distribute.experimental.MultiWorkerMirroredStrategy`.  `MultiWorkerMirroredStrategy` crea copias de todas las variables de las capas del modelo en cada dispositivo y en todos los trabajadores. Usa `CollectiveOps`, un op de TensorFlow para la comunicación colectiva, para agregar gradientes y conservar las variables sincronizadas. La guía [`tf.distribute.Strategy`](../../guide/distributed_training.ipynb) tiene más detalles sobre esta estrategia.

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

## Entrenamiento y evaluación del modelo

Posteriormente, especifique la estrategia de distribución en `RunConfig` para el estimador, y entrene y evalúe invocando `tf.estimator.train_and_evaluate`. Este tutorial distribuye sólo el entrenamiento especificando la estrategia mediante `train_distribute`. También es posible distribuir la evaluación mediante `eval_distribute`.

In [ ]:
config = tf.estimator.RunConfig(train_distribute=strategy)

classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir='/tmp/multiworker', config=config)
tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
    eval_spec=tf.estimator.EvalSpec(input_fn=input_fn)
)

## Optimizar el rendimiento del entrenamiento

Ahora dispone de un modelo y de un estimador con capacidad multitrabajador impulsado por `tf.distribute.Strategy`.  Puede probar las siguientes técnicas para optimizar el rendimiento del entrenamiento multitrabajador:

- *Aumentar el tamaño del lote:* El tamaño de lote especificado aquí es por GPU. En general, se aconseja el mayor tamaño de lote que quepa en la memoria de la GPU.

- *Casting de variables:* Hacer casting a las variables a `tf.float` si es posible. El modelo oficial de ResNet incluye [un ejemplo](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466) de cómo puede hacerse.

- *Utilice la comunicación colectiva:* `MultiWorkerMirroredStrategy` brinda múltiples [implementaciones de comunicación colectiva](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/distribute/cross_device_ops.py).

    - `RING`implementa colectivos en anillo usando gRPC como capa de comunicación entre hosts.
    - `NCCL` usa [NCCL de Nvidia](https://developer.nvidia.com/nccl) para implementar los colectivos.
    - `AUTO` deja la elección al runtime.

    La mejor elección de la implementación de colectivos depende del número y el tipo de GPU y de la interconexión de red del cluster. Para obviar la elección automática, especifique un valor válido para el parámetro `communication` del constructor de `MultiWorkerMirroredStrategy`, por ejemplo `communication=tf.distribute.experimental.CollectiveCommunication.NCCL`.

Visite la [sección de Rendimiento](../../guide/function.ipynb) de la guía para saber más sobre otras estrategias y [herramientas](../../guide/profiler.md) al alcance de su mano para optimizar el rendimiento de sus modelos TensorFlow.


## Otros ejemplos de código

1. [Ejemplo de principio a fin](https://github.com/tensorflow/ecosystem/tree/master/distribution_strategy) para entrenamiento multitrabajador en tensorflow/ecosystem usando plantillas Kubernetes. Este ejemplo comienza con un modelo Keras y lo convierte en un Estimator usando la API `tf.keras.estimator.model_to_estimator`.
2. [Modelos oficiales](https://github.com/tensorflow/models/tree/master/official), muchos de los cuales pueden configurarse para ejecutar múltiples estrategias de distribución.
